# How many zero and low emissions vehicles are registered in California?

### Load Python tools

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import geojson
import json
import jenkspy
import numpy as np
from pywaffle import Waffle
from altair import datum
import altair as alt
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
pd.options.display.float_format = '{:,.2f}'.format
alt.data_transformers.disable_max_rows()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# CARB looks for ways to meet our 5 million zero-emission vehicle target by 2030, the conversion of public and private fleets to zero emission cars and trucks offers an important opportunity to get a large number of carbon-polluting fleet vehicles off the road.
# https://ww2.arb.ca.gov/sites/default/files/2018-12/zero_emission_fleet_letter_080118.pdf

### Read data from California Department of Motor Vehicles - current as of October 2018

In [3]:
# https://data.ca.gov/dataset/vehicle-fuel-type-count-by-zip-code

In [4]:
src = pd.read_csv('https://data.ca.gov/dataset/15179472-adeb-4df6-920a-20640d02b08c/resource/4254a06d-9937-4083-9441-65597dd267e8/download/vehicle-count-as-of-1-1-2020.csv', low_memory=False)

In [5]:
df = src.copy()

### Clean up field names

In [6]:
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')\
                    .str.replace('(', '').str.replace(')', '').str.replace('-','_')

In [7]:
df.rename(columns={"zip_code": "zip"}, inplace=True)

### How many vehicles are we talking about? 

In [8]:
'{:,.0f}'.format(df.vehicles.sum())

'31,233,841'

In [9]:
df.columns

Index(['date', 'zip', 'model_year', 'fuel', 'make', 'duty', 'vehicles'], dtype='object')

### Remove unknown, out of state and heavy duty models.

In [10]:
vehicles = pd.DataFrame(df[(df['duty'] != 'Heavy') & \
                          (df['model_year'] != '<2007') &\
                          (df['zip'] != 'OOS')])

### What's left?

In [11]:
'{:,.0f}'.format(vehicles.vehicles.sum())

'19,132,886'

### How many don't rely solely on gasoline?

In [12]:
nogas = vehicles[(vehicles['fuel'] != 'Gasoline')]

In [13]:
'{:,.0f}'.format(nogas.vehicles.sum())

'2,824,578'

### Filter the data frame for fuel zero-emissions fuel types

In [14]:
cvrp_now = vehicles[(vehicles['fuel'] == 'Battery Electric') |\
               (vehicles['fuel'] == 'Hydrogen Fuel Cell') |\
                   (vehicles['fuel'] == 'Plug-in Hybrid')]

In [15]:
cvrp = vehicles[(vehicles['fuel'] == 'Battery Electric') |\
               (vehicles['fuel'] == 'Hydrogen Fuel Cell')]

### How many are zero emissions under 2035 standard?

In [16]:
'{:,.0f}'.format(cvrp.vehicles.sum())

'308,182'

In [17]:
'{:,.0f}'.format(cvrp_now.vehicles.sum())

'554,403'

### Share of newer CA vehicles that don't rely solely on gas?

In [18]:
'{:,.1f}%'.format((nogas.vehicles.sum() / vehicles.vehicles.sum())*100)

'14.8%'

### Share of CA vehicles that are zero emissions?

In [19]:
'{:,.1f}%'.format((cvrp.vehicles.sum() / vehicles.vehicles.sum())*100)

'1.6%'

In [20]:
'{:,.1f}%'.format((cvrp_now.vehicles.sum() / vehicles.vehicles.sum())*100)

'2.9%'

---

### ZIP code points

In [21]:
zips_point = gpd.read_file('/Users/mhustiles/data/data/GIS/zipcodes.geojson')

In [22]:
zips_point['zip'] = zips_point['zip'].astype(str)

In [23]:
ca_zips_point = zips_point[zips_point['state'] == 'CA']

### ZIP code boundaries

In [25]:
# Filtered CA from this national file maintained by Esri: 
# https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_ZIP_Code_Areas_anaylsis/FeatureServer/0/
zips_poly_pop = gpd.read_file('/Users/mhustiles/data/data/GIS/ca-zip-codes-esri.geojson')
zips_poly = gpd.read_file('/Users/mhustiles/data/data/GIS/ca-zip-codes-esri-demographics.geojson')

In [26]:
zips_poly.columns = zips_poly.columns.str.strip().str.lower().str.replace(' ', '_')\
                    .str.replace('(', '').str.replace(')', '').str.replace('-','_')

In [27]:
zips_poly.dropna(inplace=True)

In [28]:
zips_poly.rename(columns={"zip_code": "zip"}, inplace=True)

In [29]:
zips_poly.head(1)

,objectid,zip,po_name,pop2012,white,black,ameri_es,asian,hawn_pi,hispanic,other,mult_race,med_age,ave_hh_sz,marhh_chd,families,ave_fam_sz,hse_units,sqmi,geometry
0,4800,94002,Belmont,26832,19634,477,84,5762,221,3517,1170,1744,41.00,2.42,2925,7671,2.98,12304,5.90,"POLYGON ((-122.27703 37.53436, -122.27687 37.5..."


---

### Zip Codes with economic demographics by Esri

In [ ]:
#https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/WealthiestZipCodes2017/FeatureServer/0/
zips_wealth = gpd.read_file('/Users/mhustiles/data/github/AGStoShapefile/\
backupdir/WealthiestZipCodes2017/WealthiestZipCodesCA.geojson')

In [ ]:
zips_wealth.head(1)

In [ ]:
zips_wealth_slim = zips_wealth[['ID','NAME', 'AVGHINC_CY', 'AVGNW_CY', 'TOTPOP_CY', 'geometry']]

In [ ]:
zips_esri = zips_wealth_slim.rename(columns={"ID": "zip",
                                                    'NAME':'name',
                                 'AVGHINC_CY':'avg_house_income',
                                 'AVGNW_CY':'avg_net_worth',
                                 'TOTPOP_CY':'population', })

In [ ]:
zips_esri.head()

In [ ]:
zips_esri.plot()

---

### Group the vehicles and count them by the registration ZIP Codes

In [ ]:
zipcodes = vehicles.groupby(['zip']).agg({'vehicles':'sum'}).reset_index()

In [ ]:
zipcodes.sort_values(by='vehicles', ascending=False).head(10)
zipcodes['zip'] = zipcodes['zip'].astype(str)

### Merge the registration zip codes and merge with Esri ZIP Codes polygons

In [ ]:
zips = pd.merge(zips_esri, zipcodes, left_on='zip', right_on='zip')

In [ ]:
zips.head()

---

### Group by make. Which are most common? 

In [ ]:
# About 2.5 are listed as OTHER/UNK
make = vehicles[vehicles['make'] != 'OTHER/UNK'].groupby(['make', 'zip']).agg('sum').reset_index()

In [ ]:
most_make = make.groupby(['make']).agg('sum').reset_index().sort_values(by='vehicles', ascending=False)

In [ ]:
most_make.head()

In [ ]:
makelist = most_make.make.to_list()

In [ ]:
popular_makes = make[make['make'].isin(makelist)]

In [ ]:
make_zip = pd.DataFrame(pd.pivot_table(popular_makes, values='vehicles', \
                            index=['zip'], columns=['make'], aggfunc=np.sum, fill_value=0).reset_index())

In [ ]:
make_zip.columns = make_zip.columns.str.strip().str.lower().str.replace(' ', '_')\
                    .str.replace('(', '').str.replace(')', '').str.replace('-','_')

### Which make is most common in each ZIP code? 

In [ ]:
make_zip["total"] = make_zip.sum(axis=1)

In [ ]:
make_zip.sort_values(by='total', ascending=False).head(5)

In [ ]:
makes_list = list(make_zip.columns)

In [ ]:
makes_list

In [ ]:
make_zip[['zip', 'total']].to_csv('output/total_makes_zip.csv')

In [ ]:
make_total = pd.DataFrame(make_zip[['zip', 'total']])

In [ ]:
make_zip['winner'] = make_zip[['acura',
 'acura',
 'alfa_romeo',
 'am_general',
 'aston_martin',
 'audi',
 'bentley',
 'bmw',
 'buick',
 'cadillac',
 'chevrolet',
 'chrysler',
 'dodge',
 'ferrari',
 'fiat',
 'ford',
 'freightliner',
 'genesis',
 'gmc',
 'honda',
 'hummer',
 'hyundai',
 'infiniti',
 'jaguar',
 'jeep',
 'kia',
 'lamborghini',
 'land_rover',
 'lexus',
 'lincoln',
 'maserati',
 'mazda',
 'mercedes_benz',
 'mercury',
 'mini',
 'mitsubishi',
 'nissan',
 'pontiac',
 'porsche',
 'ram',
 'rolls_royce',
 'saturn',
 'scion',
 'smart',
 'subaru',
 'suzuki',
 'tesla',
 'toyota',
 'unk',
 'volkswagen',
 'volvo',
 'vpg',
 'workhorse']].idxmax(axis=1)

In [ ]:
make_zip.head(10)

In [ ]:
make_zip_top_ten = pd.DataFrame(make_zip[['zip','dodge','toyota','gmc','honda','ford','chevrolet','nissan',\
                               'bmw','hyundai','lexus','mercedes_benz','kia', 'total']])

In [ ]:
make_zip_top_ten.to_csv('output/make_zip_top_ten.csv')

In [ ]:
make_zip_top_ten.head()

### How many Bentleys are there? 

In [ ]:
bentley = pd.DataFrame(make_zip[['zip','bentley','total']])

In [ ]:
bentley.bentley.sum()

---

### Isolate vehicle makes to include only Teslas

In [ ]:
tesla = vehicles[(vehicles['make'] == 'TESLA')]

In [ ]:
tesla.vehicles.sum()

### Group by ZIP code and count the vehicles

In [ ]:
tesla_grouped = tesla.groupby(['zip']).agg({'vehicles':'sum'}).reset_index()

### Merge with dataframe that includes all vehicle counts by ZIP code

In [ ]:
tesla_zips = pd.merge(tesla_grouped, zips, on='zip')

### Rename the columns

In [ ]:
tesla_zips.rename(columns={'median': 'income','zip': 'zip', \
                           'vehicles_x':'teslas', 'vehicles_y':'all_vehicles'}, inplace=True)

### Normalize Tesla ownership to a rate per 1,000 vehicles

In [ ]:
tesla_zips['tesla_rate_1k'] = ((tesla_zips.teslas / tesla_zips.all_vehicles) * 1000).round(2)

In [ ]:
tesla_zips.sort_values(by='tesla_rate_1k', ascending=False).head(10)

In [ ]:
tesla_breaks \
= jenkspy.jenks_breaks(tesla_zips.tesla_rate_1k, nb_class=6)
tesla_breaks

In [ ]:
tesla_zips.to_csv('output/tesla_zips.csv')

In [ ]:
tesla_zips_slim = pd.DataFrame(tesla_zips[['zip','name','teslas','all_vehicles','tesla_rate_1k', 'avg_house_income', 'avg_net_worth']])

In [ ]:
tesla_zips_slim.sort_values(by='tesla_rate_1k', ascending=False).head(10)

### How predictive is income  

In [ ]:
tesla_zips.head()

In [ ]:
tesla_zips_corr = tesla_zips_slim[['tesla_rate_1k', 'avg_house_income']]

In [ ]:
corr = tesla_zips_corr.corr(method ='pearson')

In [ ]:
print(corr)

In [ ]:
alt.Chart(tesla_zips_slim).mark_circle(size=60).encode(
    x=alt.X('tesla_rate_1k:Q', title='Tesla rate per 1,000 vehicles', axis=alt.Axis(tickCount=6)),
    y=alt.Y('avg_net_worth:Q', title='Avg net worth', axis=alt.Axis(tickCount=5, format='$,N')),
    tooltip=['zip:N', 'teslas:O', 'avg_net_worth:Q', 'tesla_rate_1k:Q']
).properties(width=500, height=500)

In [ ]:
tesla_zips_slim.to_csv('output/tesla_zips_slim.csv')

---

### Group by model year. Which years have the most vehicles? 

In [ ]:
model_year = vehicles.groupby(['model_year']).agg('sum').reset_index()

In [ ]:
model_year.sort_values(by='model_year', ascending=False)

In [ ]:
model_year_chart = alt.Chart(model_year).mark_bar().encode(
).mark_bar().encode(
    y=alt.Y('model_year:N', title=" ", axis=alt.Axis(format='', tickCount=5)),
    x=alt.X("vehicles:Q", title="Vehicles", axis=alt.Axis(format='', tickCount=5))
)

model_year_chart_text = model_year_chart.mark_text(
    align='left',
    baseline='middle',
    dx=5
).encode(text=alt.Text('vehicles:Q', format=',')
)

(model_year_chart + model_year_chart_text).properties(height=500, \
                                                      width=700, title='California vehicles, by model year')

### Export model year table for graphics

In [ ]:
model_year.to_csv('output/model_year.csv')

---

## Fuel types

### Which types of alternative fuel models are most common?

In [ ]:
fuel = vehicles.groupby(['fuel']).agg('sum').reset_index()

In [ ]:
fuel.head(9)

In [ ]:
fuel.fuel.tolist()

In [ ]:
# Diesel and Diesel Hybrid + Flex-Fuel + Gasoline + Hybrid Gasoline
'{:,.0f}'.format((fuel.iloc[1,1] + fuel.iloc[2,1] + fuel.iloc[3,1] + fuel.iloc[4,1]))

# Chart the fuel type counts

In [ ]:
chart_fuels = alt.Chart(fuel).mark_bar().encode(
    y=alt.Y('fuel:N', title=' ',
        sort=alt.EncodingSortField(
            field="vehicles",  # The field to use for the sort
            op="sum",  # The operation to run on the field prior to sorting
            order="descending"  # The order to sort in
        ), axis=alt.Axis(format='', tickCount=5)),
    x=alt.X("sum(vehicles):Q", title="Vehicles", axis=alt.Axis(format='', tickCount=5))
)

chart_fuels_text = chart_fuels.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(text=alt.Text('sum(vehicles)', format=',')
)


(chart_fuels + chart_fuels_text)\
.properties(height=400, width=500, title='CA alternative fuel vehicles by type')

### Export fuel type count table for graphics

In [ ]:
fuel.to_csv('output/fuel.csv')

--- 

## Where are these vehicles?

In [ ]:
zip_code = vehicles.groupby(['zip']).agg('sum').reset_index()

In [ ]:
zip_code_cvrp = cvrp.groupby(['zip']).agg('sum').reset_index()

### Which ZIP codes have the most alternative fuel vehicles? (Airport areas, it seems)

In [ ]:
zip_code_cvrp.sort_values(by='vehicles',\
    ascending=False).head(10)

### Pivot on ZIP code and widen out the dataframe to count vehicle types across them

In [ ]:
sum_by_zip = pd.pivot_table(vehicles, values='vehicles', \
                            index=['zip'], columns=['fuel'], aggfunc=np.sum, fill_value=0).reset_index()

sum_by_zip.columns = sum_by_zip.columns.str.strip().str.lower().str.replace(' ', '_')\
                    .str.replace('(', '').str.replace(')', '').str.replace('-','_')

### Which type is most common in each ZIP code? 

In [ ]:
sum_by_zip["total"] = sum_by_zip.sum(axis=1)

In [ ]:
sum_by_zip.sort_values(by='total', ascending=False).head(5)

### Group the lesser-used fuel types into an 'other' category

In [ ]:
sum_by_zip['other'] = sum_by_zip.apply\
    (lambda x: x['hydrogen_fuel_cell'] + x['natural_gas'] + x['other'], axis=1)
sum_by_zip.drop(['hydrogen_fuel_cell', 'natural_gas'], axis=1, inplace=True)
sum_by_zip.drop([0], inplace=True)

### Which non-gas vehicle is most popular — the 'winner' — in each zip?

In [ ]:
sum_by_zip['winner'] = \
sum_by_zip[['hybrid_gasoline','battery_electric','diesel_and_diesel_hybrid',\
                   'flex_fuel','plug_in_hybrid', 'other']].idxmax(axis=1)

In [ ]:
sum_by_zip.sort_values(by='total', ascending=False).head(10)

### De-slugify the 'winner' category

In [ ]:
sum_by_zip['winner'] = sum_by_zip['winner'].str.replace('_', ' ', regex=False).str.capitalize()

In [ ]:
sum_by_zip['altshare'] = (((sum_by_zip['total']-sum_by_zip['gasoline'])/sum_by_zip['total'])*100).round()

In [ ]:
sum_by_zip['batteryshare'] = ((sum_by_zip['battery_electric']/sum_by_zip['total'])*100)

In [ ]:
sum_by_zip.sort_values(by='batteryshare', ascending=False).head()

### Use the "jenks" method to set fair breaks for total field

In [ ]:
breaks \
= jenkspy.jenks_breaks(sum_by_zip.batteryshare, nb_class=7)

In [ ]:
breaks

---

### Export merged polygon geodataframe as GeoJSON

In [ ]:
zips_poly.to_file('/Users/mhustiles/data/data/GIS/zips_poly.geojson', driver='GeoJSON')

In [ ]:
zips_poly_merged = zips_poly.merge(sum_by_zip, on='zip')

In [ ]:
zips_poly_merged.columns

In [ ]:
# zips_poly_merged_drop = ['objectid', 'zip', 'po_name', 'sqmi', 
#                         'battery_electric', 'diesel_and_diesel_hybrid', 'flex_fuel',
#                          'gasoline', 'hybrid_gasoline', 'other_y', 'plug_in_hybrid', 'total',
#                          'winner', 'altshare']
# zips_poly_merged.drop(zips_poly_merged_drop, inplace=True, axis=1)

In [ ]:
zips_poly_merged.to_file('/Users/mhustiles/data/data/GIS/zips_poly_merged.geojson', driver='GeoJSON')

### Export with Tesla totals

In [ ]:
tesla_zips_poly_merged = zips_poly.merge(tesla_zips_slim, on='zip')

In [ ]:
tesla_zips_poly_merged.to_file('/Users/mhustiles/data/data/fuel/tesla_zips_poly_merged.geojson', driver='GeoJSON')

In [ ]:
tesla_zips_poly_merged.head()

### Merge ZIP points with Tesla & electric totals

In [ ]:
teslas_point_merged = ca_zips_point.merge(tesla_zips_slim, on='zip')

In [ ]:
teslas_point_merged.sort_values(by='tesla_rate_1k', ascending=False).head(10)

In [ ]:
teslas_point_merged.to_file('output/teslas_point_merged.geojson', driver='GeoJSON')

### Merge ZIP points with all vehicle totals

In [ ]:
ca_zips_point_merged = ca_zips_point.merge(sum_by_zip, on='zip')

In [ ]:
ca_zips_point_merged.plot()

### Export merged points geodataframe as GeoJSON

In [ ]:
ca_zips_point_merged.head(5)

In [ ]:
ca_zips_point_merged.to_file('/Users/mhustiles/data/data/fuel/ca_zips_point_merged.geojson', driver='GeoJSON')

### Convert polygons to mbtiles for Mapbox. Export.

In [ ]:
!tippecanoe --force -r1 -pk -pf -Z5 -z13 -o \
/Users/mhustiles/data/data/fuel/ca_zips_point_merged.mbtiles \
/Users/mhustiles/data/data/fuel/ca_zips_point_merged.geojson

### Convert points to mbtiles for Mapbox. Export.

In [ ]:
!tippecanoe --force -r1 -pk -pf -Z5 -z13 -o \
/Users/mhustiles/data/data/fuel/ca_zips_point_merged.mbtiles \
/Users/mhustiles/data/data/fuel/ca_zips_point_merged.geojson

### Convert Tesla polygons to mbtiles for Mapbox. Export.

In [ ]:
!tippecanoe --force -r1 -pk -pf -Z5 -z13 -o \
/Users/mhustiles/data/data/fuel/tesla_zips_poly_merged.mbtiles \
/Users/mhustiles/data/data/fuel/tesla_zips_poly_merged.geojson

---

## Waffle plot

In [ ]:
fuel.head(9)

In [ ]:
fuel.vehicles.sum()

In [ ]:
fuel.iloc[0,1] + fuel.iloc[5,1] + fuel.iloc[6,1] + fuel.iloc[7,1]

In [ ]:
fuel['per_1000'] =  ((fuel['vehicles'] / fuel.vehicles.sum()) * 1000).round(2)

In [ ]:
fuel

In [ ]:
fig = plt.figure(
    FigureClass=Waffle, 
    rows=21,
    values=fuel.per_1000,
    labels=list(fuel.fuel),
    figsize=(30,20),
    icons='car-side',
    colors=['#ec8431','#e6e6e6','#e6e6e6','#e6e6e6','#e6e6e6','#e6e6e6','#e6e6e6', '#e6e6e6', '#e6e6e6'],
    legend={
        'loc': 'lower left',
        'bbox_to_anchor': (0, -0.4),
        'ncol': len(fuel),
        'framealpha': 0,
        'fontsize': 0
    }
)

See related [Twitter thread](https://twitter.com/stiles/status/1193416749116358656)